In [42]:
import numpy as np
from numpy import std
from numpy import mean
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [40]:
df = pd.read_csv("sql_df_preprocessed.csv")
df = df.drop(["full_url", "parsed_url", "event_time", "hostname"], axis=1)
df

,sql_keyword,params_length,request_volume,special_chars,values_to_length,injection
0,8,178,3.5,11,10.674157,1
1,0,5,3.5,1,20.000000,1
2,0,13,3.5,5,38.461538,1
3,0,21,3.5,3,14.285714,1
4,1,24,3.5,3,16.666667,1
...,...,...,...,...,...,...
6126,1,48,0.5,1,4.166667,0
6127,0,34,0.5,1,2.941176,0
6128,0,37,0.5,1,2.702703,0
6129,0,32,0.5,1,3.125000,0


In [49]:
# load data set and split it into metrics and classifier
np_dataset = df.to_numpy()
X = df.iloc[:, :-1].values
y = df. iloc [:, -1].values

print(X.shape)
print(y.shape)

(6131, 5)
(6131,)


In [65]:
round = 1

# init cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=1)
# split into folds
for train, test in kfold.split(np_dataset):
    print(f'Round number {round}')
    X_train = np_dataset[train][:, :-1]
    y_train = np_dataset[train][:, -1]

    X_test = np_dataset[test][:, :-1]
    y_test = np_dataset[test][:, -1]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    neighbours = range(2,6)
    for i in neighbours:
        print(f'Neighbours number {i}')
        # create model and train it
        knn = KNeighborsClassifier(n_neighbors=i)
        knn.fit(X_train, y_train)

        # Test results
        y_pred = knn.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print("Accuracy:", accuracy)
    round = round + 1

Round number 1
Neighbours number 2
Accuracy: 1.0
Neighbours number 3
Accuracy: 1.0
Neighbours number 4
Accuracy: 1.0
Neighbours number 5
Accuracy: 1.0
Round number 2
Neighbours number 2
Accuracy: 1.0
Neighbours number 3
Accuracy: 1.0
Neighbours number 4
Accuracy: 1.0
Neighbours number 5
Accuracy: 1.0
Round number 3
Neighbours number 2
Accuracy: 0.9967373572593801
Neighbours number 3
Accuracy: 0.9967373572593801
Neighbours number 4
Accuracy: 0.9951060358890701
Neighbours number 5
Accuracy: 0.9951060358890701
Round number 4
Neighbours number 2
Accuracy: 0.99836867862969
Neighbours number 3
Accuracy: 0.99836867862969
Neighbours number 4
Accuracy: 0.99836867862969
Neighbours number 5
Accuracy: 0.99836867862969
Round number 5
Neighbours number 2
Accuracy: 0.99836867862969
Neighbours number 3
Accuracy: 0.99836867862969
Neighbours number 4
Accuracy: 0.99836867862969
Neighbours number 5
Accuracy: 0.9951060358890701
Round number 6
Neighbours number 2
Accuracy: 0.9967373572593801
Neighbours numb